In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from lmfit.models import GaussianModel,LinearModel
from glob import glob
from lmfit import models

/home/frederik/anaconda3/lib/python3.4/site-packages/IPython/html.py:14: ShimWarning: The `IPython.html` package has been deprecated. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)


In [2]:
%matplotlib qt 

#Rutherfordstreuung

##Rutherfordsteuuformel
<br>
$\huge \huge \frac{d\sigma}{d\Omega} = 1.3 \cdot 10^{-3} \left( \frac{Z_1 Z_2}{E_0} \right) \frac{1}{sin^4\left(\frac{\theta}{2}\right)}$

<ul>
    <li>$\sigma$: Differentielle WQ
    <li>$\Theta$: Winkel
    <li>$Z_i$: Kernlagundszahl
    <li>E: Energie
</ul>   

In [3]:
def rutherfordsteuung(theta,Z_1,Z_2,E,b):
    e = 1.602 * 10**-19
    return (Z_1*Z_2*e**2)/(4*E*np.sin((3.14*theta-b)/360)**4)

# Energieverlust von $\alpha$-Strahlung in Luft

# Energie-Kanal kalibration

In [4]:
cd /home/frederik/Dokumente/FP/Rutherford/MessdatenRutherford/kalib/

/home/frederik/Dokumente/FP/Rutherford/MessdatenRutherford/kalib


In [5]:
data = np.array(pd.read_csv("energiekalpib_30Torr.csv",skiprows=15,delimiter="\t")).T

In [6]:
plt.errorbar(data[0],data[1],np.sqrt(data[1]))
plt.show()

## Händische schätzung für die Peaks

    - 1: x=250, y=180
    - 2: x=320, y=266
    - 3: x=373, y=158
    - 4: x=551, y=159
    
Mulitgauss mit den Werten konfigurieren und Fehler berechnen

In [7]:
err = np.sqrt(data[1][200:612])
err = np.where(err == 0, 1, err)

In [8]:
y = data[1][200:612]
x = data[0][200:612]


g_1 = GaussianModel(prefix="g_1")
pars = g_1.guess(y,x=x)

pars['g_1center'].set(260, min=240, max=275)
pars['g_1sigma'].set(15, min=3,max=22)
pars['g_1amplitude'].set(10000, min=90)

g_2 = GaussianModel(prefix="g_2")
pars.update( g_2.make_params())

pars['g_2center'].set(320, min=300, max=345)
pars['g_2sigma'].set(15, min=3)
pars['g_2amplitude'].set(20000, min=1)


g_3 = GaussianModel(prefix="g_3")
pars.update( g_3.make_params())

pars['g_3center'].set(373, min=350, max=400)
pars['g_3sigma'].set(15, min=3,max=20)
pars['g_3amplitude'].set(10000, min=1)

g_4 = GaussianModel(prefix="g_4")
pars.update( g_4.make_params())

pars['g_4center'].set(551, min=530, max=570)
pars['g_4sigma'].set(15, min=3)
pars['g_4amplitude'].set(20000, min=1)



mod = g_1 + g_2 + g_3 + g_4


out = mod.fit(y,pars,x=x,weights=1/err)

In [9]:
print(out.fit_report())

[[Model]]
    (((Model(gaussian, prefix='g_1') + Model(gaussian, prefix='g_2')) + Model(gaussian, prefix='g_3')) + Model(gaussian, prefix='g_4'))
[[Fit Statistics]]
    # function evals   = 166
    # data points      = 412
    # variables        = 12
    chi-square         = 686.653
    reduced chi-square = 1.717
[[Variables]]
    g_1sigma:       22         +/- 0.251185 (1.14%) (init= 15)
    g_1amplitude:   8700.66363 +/- 308.8438 (3.55%) (init= 10000)
    g_1center:      255.238186 +/- 0.978131 (0.38%) (init= 260)
    g_1fwhm:        51.8060400 +/- 0.591496 (1.14%)  == '2.3548200*g_1sigma'
    g_2sigma:       19.4956988 +/- 0.752410 (3.86%) (init= 15)
    g_2amplitude:   12483.3777 +/- 459.5527 (3.68%) (init= 20000)
    g_2center:      315.461920 +/- 0.556987 (0.18%) (init= 320)
    g_2fwhm:        45.9088615 +/- 1.771791 (3.86%)  == '2.3548200*g_2sigma'
    g_3amplitude:   7156.31880 +/- 243.6512 (3.40%) (init= 10000)
    g_3center:      372.541213 +/- 0.814979 (0.22%) (init= 373)
 

In [10]:
plt.errorbar(data[0][200:612],data[1][200:612],np.sqrt(data[1][200:612]),fmt='. b',label="Messdaten")
plt.plot(data[0][200:612],out.best_fit, '-r',label="Muli-Gauss-Fit")
plt.xlabel("Kanal",fontsize=22)
plt.ylabel('Counts',fontsize=22)
plt.xticks(fontsize=18)
plt.yticks(fontsize=18)
plt.legend()
plt.show()

## Mittelwerte den Energien zu ordnen
    - Peak  Energie  Kanal
    -   1    4,871   255+1
    -   2    5.49    315+0.6
    -   3    6.00    372+0.8
    -   4    7.69    549+0.3

In [11]:
eich_kanal = np.array([255,315,372,549])
eich_energie = np.array([4.871,5.49,6.00,7.69])
eich_errx = np.array([1,0.6,0.8,0.3])

In [12]:
mod_eich = LinearModel()
guess = mod_eich.guess(eich_energie,x=eich_kanal)
out_eich = mod_eich.fit(eich_energie,guess,x=eich_kanal)
print(out_eich.fit_report())
out_eich.plot()
plt.legend(loc='best')
plt.show()

[[Model]]
    Model(linear)
[[Fit Statistics]]
    # function evals   = 3
    # data points      = 4
    # variables        = 2
    chi-square         = 0.001
    reduced chi-square = 0.001
[[Variables]]
    slope:       0.00953604 +/- 0.000111 (1.16%) (init= 0.009536043)
    intercept:   2.45818986 +/- 0.042991 (1.75%) (init= 2.45819)
[[Correlations]] (unreported correlations are <  0.100)
    C(slope, intercept)          = -0.959 


In [13]:
plt.errorbar(eich_kanal,eich_energie,xerr=eich_errx,fmt='.r',label="Messdaten")
plt.plot(eich_kanal,out_eich.best_fit,label="linearer Fit")
plt.xlabel("Kanal",fontsize=22)
plt.ylabel('Energie [MeV]',fontsize=22)
plt.xticks(fontsize=18)
plt.yticks(fontsize=18)
plt.xlim([240,560])
plt.ylim([4.7,7.9])
plt.legend(loc="best")
plt.show()

# Fitten der Mulitgauß bei unterschiedlichen Drücken

In [14]:
y = data[1][200:612]
x = data[0][200:612]


g_1 = GaussianModel(prefix="g_1")
pars = g_1.guess(y,x=x)

pars['g_1center'].set(181)
pars['g_1sigma'].set(15, min=3)
pars['g_1amplitude'].set(10000, min=90)

g_2 = GaussianModel(prefix="g_2")
pars.update( g_2.make_params())

pars['g_2center'].set(239)
pars['g_2sigma'].set(15, min=3)
pars['g_2amplitude'].set(10000, min=1)


g_3 = GaussianModel(prefix="g_3")
pars.update( g_3.make_params())

pars['g_3center'].set(292)
pars['g_3sigma'].set(15, min=3)
pars['g_3amplitude'].set(10000, min=1)

g_4 = GaussianModel(prefix="g_4")
pars.update( g_4.make_params())

pars['g_4center'].set(457)
pars['g_4sigma'].set(15, min=3)
pars['g_4amplitude'].set(10000, min=1)



mod = g_1 + g_2 + g_3 + g_4

In [15]:
i = liste[0]
out_2 = mod.fit(i[1][45:-5],pars,x=i[0][45:-5])
out_2.plot()
plt.show()

NameError: name 'liste' is not defined

In [16]:
print(out_2.values['g_1center'])
print(out_2.values['g_2center'])
print(out_2.values['g_3center'])
print(out_2.values['g_4center'])

NameError: name 'out_2' is not defined

In [ ]:
liste_fit_params_4.append([])
liste_fit_params_4[-1].append(out_2.best_values["g_4center"])
liste_fit_params_4[-1].append(out_2.best_values["g_4amplitude"])
liste_fit_params_4[-1].append(out_2.best_values["g_4sigma"])

# Beth-Bloch 

In [17]:
np_liste_fit_1 = np.loadtxt("/home/frederik/Dokumente/FP/Rutherford/MessdatenRutherford/kalib/paras_1").T
np_liste_fit_2 = np.loadtxt("/home/frederik/Dokumente/FP/Rutherford/MessdatenRutherford/kalib/paras_2").T
np_liste_fit_3 = np.loadtxt("/home/frederik/Dokumente/FP/Rutherford/MessdatenRutherford/kalib/paras_3").T
np_liste_fit_4 = np.loadtxt("/home/frederik/Dokumente/FP/Rutherford/MessdatenRutherford/kalib/paras_4").T

In [18]:
dE_1 = (np_liste_fit_1[0][:-1] - np_liste_fit_1[0][1:])*0.0095
dE_2 = (np_liste_fit_2[0][:-1] - np_liste_fit_2[0][1:])*0.0095
dE_3 = (np_liste_fit_3[0][:-1] - np_liste_fit_3[0][1:])*0.0095
dE_4 = (np_liste_fit_4[0][:-1] - np_liste_fit_4[0][1:])*0.0095

In [19]:
x = (6/760)*np.arange(100,725,25)
dx = (x[:-1]-x[1:])

In [20]:
dE_dx_1 = dE_1/dx[:dE_1.size]
dE_dx_2 = dE_2/dx[:dE_2.size]
dE_dx_3 = dE_3/dx[:dE_3.size]
dE_dx_4 = dE_4/dx[:dE_4.size]

In [21]:
E_1 = abs(np_liste_fit_1[0][0:-1] + np_liste_fit_1[0][1:])*0.0095/2
E_2 = abs(np_liste_fit_2[0][0:-1] + np_liste_fit_2[0][1:])*0.0095/2
E_3 = abs(np_liste_fit_3[0][0:-1] + np_liste_fit_3[0][1:])*0.0095/2
E_4 = abs(np_liste_fit_4[0][0:-1] + np_liste_fit_4[0][1:])*0.0095/2

In [117]:
plt.plot(E_1,-dE_dx_1)
plt.plot(E_2,-dE_dx_2)
plt.plot(E_3,-dE_dx_3)
plt.plot(E_4,-dE_dx_4)
plt.show()

## Bethe-Bloch $\sim \frac{A}{\beta} \left[ ln(B \cdot \beta) - \beta \right]$

In [119]:
def bethe_func(x,A,B,C):
    return (A*np.log(B*x)-x)/x-C
test = bethe_func(x,1.8,11,0)

In [120]:
x = np.arange(0.1,100,1)
plt.plot(x,test)
plt.show()

In [24]:
mod_ln = models.Model(ln) 
mod_const = models.ConstantModel()
guess_ln = mod_const.guess(dE_dx_1,x=dE_1)
guess_ln.update(mod_ln.make_params())

ln_model = mod_ln + mod_const

print(ln_model.param_names)

guess_ln['c'].set(0.0001)

{'A', 'c'}


In [25]:
bethe_1 = ln_model.fit(dE_dx_1,paras,x=E_1)

NameError: name 'paras' is not defined

In [26]:
print(bethe_1.fit_report())

NameError: name 'bethe_1' is not defined

In [29]:
plt.plot(np.arange(0,5,0.1),-ln(np.arange(0,5,0.1),0.001))
plt.show()